# Extract Simons Observatory instrument parameters

In order to be independent of `sotodlib` and track versions of the instrument model parameters more easily, we extract them to a file included in this repository.

The versioning will be YYYY.MM

In [1]:
import numpy as np

In [2]:
import sotodlib.sim_hardware as hardware

hw = hardware.get_example()

In [3]:
import sotodlib

In [4]:
sotodlib.__version__

'0.3.0+6.g0c64e07'

In [5]:
import h5py

In [6]:
instrument_parameters = h5py.File("simonsobs_instrument_parameters_2020.06.h5", "w")

instrument_parameters.attrs["sotodlib_version"] = sotodlib.__version__

In [7]:
fwhm_arcmin = {"SA" : hw.data["telescopes"]["SAT1"]["fwhm"],
"LA":hw.data["telescopes"]["LAT"]["fwhm"]}

In [8]:
bands = ("LF1", "LF2", "MFF1", "MFF2", "MFS1", "MFS2", "UHF1", "UHF2")
indices = (0, 1, 2, 3, 2, 3, 4, 5)

noise_band_index = dict(zip(bands, indices))

In [9]:
for tube_id, (tube, tube_properties) in enumerate(hw.data["tubes"].items()):
    bands = [tube_properties["type"] + num for num in "12"]
    telescope = tube[0] + "A"
    for i_band, band in enumerate(bands):
        tag = tube + "_" + band
        print(tag)
        ch = instrument_parameters.create_group(tag)
        ch.attrs["telescope"] = telescope
        ch.attrs["band"] = band
        ch.attrs["tube"] = tube
        ch.attrs["tube_id"] = tube_id
        ch.attrs["band_id"] = i_band
        ch.attrs["noise_band_index"] = noise_band_index[band]
        ch.attrs["fwhm_arcmin"] = fwhm_arcmin[telescope][band]
        properties = hw.data["bands"][band]
        ch.attrs["center_frequency_GHz"] = properties["center"]
        ch.create_dataset("bandpass_frequency_GHz", data=np.linspace(properties["low"], properties["high"], 10))
        ch.create_dataset("bandpass_weight", data=np.ones(10, dtype=np.float32))

LT0_UHF1
LT0_UHF2
LT1_UHF1
LT1_UHF2
LT2_MFF1
LT2_MFF2
LT3_MFF1
LT3_MFF2
LT4_MFS1
LT4_MFS2
LT5_MFS1
LT5_MFS2
LT6_LF1
LT6_LF2
ST0_UHF1
ST0_UHF2
ST1_MFF1
ST1_MFF2
ST2_MFS1
ST2_MFS2
ST3_LF1
ST3_LF2


In [10]:
instrument_parameters.close()